## ** BookLed Experiment 4 (AI Book with voiceover and soundtrack) - ITALIAN **
--------------------------------------------------------------------------------
### **In this experiment, when you turn the pages of the physical book, an MP3 audio is played whose content is generated by AI in real time.**
#### The AI ​​analyzes the graphic content of the pages of the book and **produces a story based on what it sees** represented in the drawings.
#### By turning pages, the corresponding images are provided to the AI ​​via a chat completion API.
#### In this way, by turning the pages of the physical paper book, the story is told by the AI ​​and each time the story is different.
#### Then the voice is obtained by invoking OpenAI's **TTS API**.
#### With the prompt provided, the narrator's voice is reproduced in Italian.
#### In this experiment, in addition to the narrative voice, a **soundtrack contextual to the page read** is also reproduced.
#### Using **chat completion**, in this example the AI ​​does not maintain the context of the story it is producing.
01/06/2024 V5

# Follow this steps
- Connect the BookLed to the PC through USB
- Be sure to configure the right COM port number
- Be sure to have an active internet connection: the OpenAI API will be canned from the internet
- Be sure to have an active OpenAI APi account
- Power ON the Bookled pushing the button switch
- Run the next cell of this notebook

In [ ]:
# Windows 10 tips:
#	to make it work you have to do this:
# pip install ffmpeg-downloader
# ffdl install --add-path
# Installing collected FFmpeg binaries: 6.1.1@essentials
# Successfully installed FFmpeg binaries: 6.1.1@essentials in
#    C:\Users\gravi\AppData\Local\ffmpegio\ffmpeg-downloader\ffmpeg\bin
#
# to make it work then copy
#    ffmpeg.exe e ffprobe.exe dentro al venv di python
#
# see https://stackoverflow.com/questions/74651215/couldnt-find-ffmpeg-or-avconv-python

In [ ]:
import os
import time
import serial
import threading
import requests
from pydub import AudioSegment
import wave
import pyaudio
from dotenv import load_dotenv
import json
import openai

# Ignore DeprecationWarning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Create a temporary directory to save downloaded MP3 files
temp_dir = "temp_mp3"
os.makedirs(temp_dir, exist_ok=True)

# Configure BookLed serial connection

In [ ]:
######################################################################################################
# Configure BookLed serial connection (modify 'COM4' with the correct COM port)
######################################################################################################
ser = serial.Serial(
    port='COM4',
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1
)

# Load the OpenAI API key

In [ ]:
######################################################################################################
# Load OpenAI API key
######################################################################################################
# Load environment variables from .env file
load_dotenv()

# Read OpenAI API from .env file
api_key = os.getenv('OPENAI_API_KEY')
#print("openAI apy key=",api_key)

# Set the OpenAI API key
openai.api_key = api_key
client = openai

# Prompts and RAG contents

In [ ]:
######################################################################################################
# Set remote and local contents
######################################################################################################

# URL of GitHub prompt file
prompt_url = "https://github.com/robotoons/BookLed/raw/main/codes/jupyter%20notebooks/ai/prompts/Gu-File%2001%20chatbot%20reader%20prompt.txt"
image_url = "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ENG/BL01-ENG-PAGE03.jpg"

# local prompt filename
reader_chatbot_prompt_filename = "data/Gu-File 01 chatbot reader prompt.txt"

# URL of GitHub images for RAG (italian images)
image_urls = [
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE01.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE02.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE03.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE04.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE05.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE06.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE07.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE08.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE09.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE10.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE11.jpg",
    "https://raw.githubusercontent.com/robotoons/BookLed/main/codes/jupyter%20notebooks/ai/rag%20docs/Gu-File%2001/images/ITA/BL01-ITA-PAGE12.jpg"
]

# URL of MP3 soundtracks 
soundtrack_urls = [
    "https://github.com/robotoons/BookLed/blob/main/contents/Gu-File_%2301/mp3/Soundtrack/tav1_4.mp3?raw=true",
    "https://github.com/robotoons/BookLed/blob/main/contents/Gu-File_%2301/mp3/Soundtrack/tav5_8.mp3?raw=true",
    "https://github.com/robotoons/BookLed/blob/main/contents/Gu-File_%2301/mp3/Soundtrack/tav9_10.mp3?raw=true",
    "https://github.com/robotoons/BookLed/raw/main/contents/Gu-File_%2301/mp3/Soundtrack/tav11.mp3?raw=true",
    "https://github.com/robotoons/BookLed/blob/main/contents/Gu-File_%2301/mp3/Soundtrack/tav12.mp3?raw=true"
]

# Mp3 soundtrack file map
soundtrack_map = {
    range(1, 5): "tav1_4.mp3",
    range(5, 9): "tav5_8.mp3",
    range(9, 11): "tav9_10.mp3",
    range(11, 12): "tav11.mp3",
    range(12, 13): "tav12.mp3"
}

# Functions

In [ ]:
######################################################################################################
# Read BookLed page function
######################################################################################################
def read_last_page_number():
    """
    This function reads data from the serial port and extracts the last
    page number from the received data. It handles cases where multiple
    page numbers may be sent in a single read.
    """    
    while True:
        # Read a line from the serial port until a newline character is found
        line = ser.read_until().decode('utf-8').strip()
        #print(f"Read line from serial: {line}")

        # Remove any carriage return and line feed characters for clean debugging
        clean_line = line.replace('\r', '').replace('\n', '')
        
        # Split the cleaned line by the delimiter 'P:' to find all page codes
        pages = clean_line.split('P:')
        last_page = None

        # Iterate through the list of possible page codes
        for page in pages:
            if page:
                try:
                    # Strip any extra whitespace and convert the hex value to decimal
                    page_number_hex = page.strip()
                    page_number_dec = int(page_number_hex, 16)
                    # Update the last_page with the most recent valid page number
                    last_page = page_number_dec
                    #print(f"Extracted page number: {last_page}")
                except (ValueError):
                    # Print an error message if conversion fails and continue
                    print("Error: invalid value")
                    continue
                    
        # Return the last valid page number found
        if last_page is not None:
            return last_page



######################################################################################################
# read file to string function
######################################################################################################
def read_file_to_string(file_path):
    try:
        # Open the file in read mode
        with open(file_path, 'r', encoding='utf-8') as file:
            # Read file content
            content = file.read()
            return content
    except FileNotFoundError:
        print("File not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


######################################################################################################
# download file from github function
######################################################################################################
def download_file_from_github(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Controlla se ci sono stati errori nella richiesta
        return response.text
    except requests.RequestException as e:
        print(f"Error downloading file from {url}: {e}")
        return None


######################################################################################################
# download mp3 file from url function
######################################################################################################
def download_mp3(url, filename):
    print(f"Downloading MP3 from {url}")
    response = requests.get(url)
    file_path = os.path.join(temp_dir, filename)
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded MP3 to {file_path}")
    return file_path


######################################################################################################
# play mp3 function (thread and volume control)
######################################################################################################
import os
import threading
import pyaudio
import wave
from pydub import AudioSegment
import uuid

def play_audio(file_path, stop_flag, unique_id, volume):
    print(f"Starting playback of {file_path}")
    
    # Convert MP3 to WAV with a unique filename
    audio = AudioSegment.from_mp3(file_path)
    audio = audio + volume  # Adjust the volume
    temp_wav_path = f"temp_{unique_id}.wav"
    audio.export(temp_wav_path, format="wav")
    print(f"Converted MP3 to WAV for {file_path}")

    # Open the unique WAV file
    wf = wave.open(temp_wav_path, 'rb')

    # Create a PyAudio instance
    p = pyaudio.PyAudio()

    # Open a stream
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Read the data
    data = wf.readframes(1024)

    # Play the stream
    while data and not stop_flag.is_set():
        stream.write(data)
        data = wf.readframes(1024)

    # Stop the stream
    stream.stop_stream()
    stream.close()

    # Close PyAudio and the WAV file
    p.terminate()
    wf.close()
    os.remove(temp_wav_path)  # Remove the unique temp file
    print(f"Playback finished for {file_path}")

def play_mp3_in_thread(file_path, stop_flag, volume):
    unique_id = uuid.uuid4()  # Generate a unique identifier for each thread
    thread = threading.Thread(target=play_audio, args=(file_path, stop_flag, unique_id, volume), name=file_path)
    thread.start()
    return thread


######################################################################################################
# chat completions OpenAI API call function
######################################################################################################
def call_openai_api(prompt_text, image_url):
    response = client.chat.completions.with_raw_response.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                        "detail": "high"
                    },
                },
            ],
        }
    ],
    max_tokens=400,
    )
    print(f"time: {response.elapsed.total_seconds():.2f}s")
    
    # Parse JSON data
    #print(response.http_response.text)
    json_data = json.loads(response.http_response.text)
    
    # Extract the "content" string
    content = json_data["choices"][0]["message"]["content"]
    
    return content


######################################################################################################
# TTS text to speech OpenAI API call function
######################################################################################################
def text_to_speech(content):
    start_time = time.time()
    speech_file_path = os.path.join(temp_dir, "tts.mp3")   
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=content
    )
    response.stream_to_file(speech_file_path)
    #with open(speech_file_path, 'wb') as f:
    #   f.write(response.content)
    
    print(f"time: {(time.time() - start_time):.2f}s")
    
    return speech_file_path

# Select the source of the prompt file (local or github)

In [ ]:
###########################################
# SELECT PROMPT FROM LOCAL FILE OR GITHUB #
###########################################
if(0):
    # Download prompt from GitHub
    response = requests.get(prompt_url)
    prompt_text = response.text
else:
    # Read the local prompt file 
    prompt_text = read_file_to_string(reader_chatbot_prompt_filename)

# Play with BookLed and AI with voiceover and soundtrack

In [ ]:
# set mix audio levels
VOICEOVER_VOLUME = 0
SOUNDTRACK_VOLUME = -15

# initialize global variables
current_page = None
stop_audio = False
audio_thread = None
stop_audio_flag = threading.Event()
soundtrack_thread = None
stop_soundtrack_flag = threading.Event()

# start the play loop
print("BookLed play start!")
try:
    while True:
        page_number = read_last_page_number()
        if page_number != current_page:
            current_page = page_number
            print("BookLed page number=", page_number)

            # Controllo per il file soundtrack MP3
            soundtrack_file = None
            for page_range, soundtrack in soundtrack_map.items():
                if page_number in page_range:
                    soundtrack_file = soundtrack
                    break

            if 1 <= current_page <= 12:

                # Controllo per continuare o interrompere la riproduzione del soundtrack
                if soundtrack_file:
                    soundtrack_index = list(soundtrack_map.values()).index(soundtrack_file)
                    if soundtrack_thread is None or not soundtrack_thread.is_alive():
                        soundtrack_path = download_mp3(soundtrack_urls[soundtrack_index], soundtrack_file)
                        stop_soundtrack_flag.clear()
                        print("Start soundtrack MP3 audio play")
                        soundtrack_thread = play_mp3_in_thread(soundtrack_path, stop_soundtrack_flag, volume=SOUNDTRACK_VOLUME)
                    elif soundtrack_file not in soundtrack_thread.name:
                        stop_soundtrack_flag.set()
                        soundtrack_thread.join()
                        soundtrack_path = download_mp3(soundtrack_urls[soundtrack_index], soundtrack_file)
                        stop_soundtrack_flag.clear()
                        print("Start soundtrack MP3 audio play")
                        soundtrack_thread = play_mp3_in_thread(soundtrack_path, stop_soundtrack_flag, volume=SOUNDTRACK_VOLUME)

                # interrompi la riproduzione dell'audio voiceover
                print("Stop voice-over MP3 audio play")
                stop_audio = True
                if audio_thread is not None:
                    stop_audio_flag.set()
                    audio_thread.join()
                #stop_audio = False

                image_url = image_urls[current_page - 1]
                print("OpenAI call #1: convert image in storytelling")
                content = call_openai_api(prompt_text, image_url)
                print("OpenAI call #2: convert text in speech")
                speech_file_path = text_to_speech(content)
                print("Start voice-over MP3 audio play")
                stop_audio_flag.clear()
                audio_thread = play_mp3_in_thread(speech_file_path, stop_audio_flag, volume=VOICEOVER_VOLUME)

            else:
                stop_audio = True
                if audio_thread is not None:
                    audio_thread.join()
                stop_soundtrack_flag.set()
                if soundtrack_thread is not None:
                    soundtrack_thread.join()
                audio_thread = None
                soundtrack_thread = None
except KeyboardInterrupt:
    print("Program interrupted")
finally:
    ser.close()
    stop_audio = True
    if audio_thread is not None:
        audio_thread.join()
    stop_soundtrack_flag.set()
    if soundtrack_thread is not None:
        soundtrack_thread.join()
    print("Program terminated")
